#실내 중소형 주차장 데이터분석

1. 로컬에서 파일 가져오기

powerShell에서 
scp -r [로컬파일경로] elicer@14.35.173.14:[ssh파일경로]
로 진행

2. class 파악

In [1]:
import os
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jm/eda/대형주차장'

# 모든 JSON 파일에서 label과 class 정보를 추출하는 함수
def count_labels_and_classes(json_folder_path):
    bbox2d_class_count = {}
    segmentation_class_count = {}

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # bbox2d 내의 "name"을 추출하여 bbox2d_class_count에 저장
                    for bbox in data.get("bbox2d", []):
                        class_name = bbox.get("name")
                        if class_name:
                            bbox2d_class_count[class_name] = bbox2d_class_count.get(class_name, 0) + 1

                    # segmentation 내의 "name"을 추출하여 segmentation_class_count에 저장
                    for seg in data.get("segmentation", []):
                        class_name = seg.get("name")
                        if class_name:
                            segmentation_class_count[class_name] = segmentation_class_count.get(class_name, 0) + 1

    return bbox2d_class_count, segmentation_class_count

# 함수 호출 및 결과 출력
bbox2d_class_count, segmentation_class_count = count_labels_and_classes(json_folder_path)

# 각 class별로 빈도 출력
print("bbox2d Classes and Counts:")
for class_name, count in bbox2d_class_count.items():
    print(f'Class: {class_name}, Count: {count}')

print("\nsegmentation Classes and Counts:")
for class_name, count in segmentation_class_count.items():
    print(f'Class: {class_name}, Count: {count}')

# 총 클래스 수 출력
print(f"\nTotal bbox2d Classes: {len(bbox2d_class_count)}")
print(f"Total segmentation Classes: {len(segmentation_class_count)}")


bbox2d Classes and Counts:
Class: Car, Count: 170200
Class: Other Vehicle, Count: 17695
Class: Motorbike, Count: 1552
Class: No Parking Stand, Count: 2444
Class: Van, Count: 4941
Class: Movable Obstacle, Count: 3345
Class: Adult, Count: 5744
Class: Bicycle, Count: 1230
Class: Speed Bump, Count: 3537
Class: Child, Count: 577
Class: Planter Barrier, Count: 1221
Class: Traffic Pole, Count: 5222
Class: Disabled Parking Space, Count: 975
Class: Parking Block, Count: 16645
Class: Parking Sign, Count: 240
Class: Traffic Light, Count: 54
Class: Gate Arm, Count: 349
Class: Parking Meter, Count: 30
Class: Barrier Gate, Count: 330
Class: Cylindrical Bollard, Count: 723
Class: U-shaped Bollard, Count: 1331
Class: Water Container, Count: 75
Class: Electric Scooter, Count: 64
Class: Waste Tire, Count: 836
Class: Other Parking Space, Count: 133
Class: Other Road Barriers, Count: 2582
Class: Traffic Barricade, Count: 282
Class: Traffic Cone, Count: 3767
Class: Adjustable Parking Pole, Count: 93
Class:

3. segmentation 추출

In [2]:
import os
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jm/eda/대형주차장'
output_folder_path = '/home/elicer/jm/seg/대형주차장'

# 새로운 JSON 파일로 저장하는 함수 및 크기 비교
def save_segmentation_only_json(json_folder_path, output_folder_path):
    # 출력 폴더가 없으면 생성
    os.makedirs(output_folder_path, exist_ok=True)

    original_total_size = 0
    reduced_total_size = 0

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                original_total_size += os.path.getsize(file_path)  # 원본 파일 크기 합산

                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # segmentation이 있는 경우에만 파일 생성
                    if "segmentation" in data and data["segmentation"]:
                        # 불필요한 항목들 제거
                        data.pop("bbox2d", None)
                        data.pop("calib", None)
                        data.pop("bbox3d", None)

                        # 새로운 JSON 파일로 저장
                        relative_path = os.path.relpath(root, json_folder_path)
                        output_dir = os.path.join(output_folder_path, relative_path)
                        os.makedirs(output_dir, exist_ok=True)
                        output_file_path = os.path.join(output_dir, file)
                        with open(output_file_path, 'w') as output_file:
                            json.dump(data, output_file, indent=4)
                        
                        print(f'파일 생성됨: {output_file_path}')
                        reduced_total_size += os.path.getsize(output_file_path)  # 변환된 파일 크기 합산

    return original_total_size, reduced_total_size

# 함수 호출
original_size, reduced_size = save_segmentation_only_json(json_folder_path, output_folder_path)

# 크기 비교 출력
reduction_percentage = 100 * (original_size - reduced_size) / original_size if original_size > 0 else 0
print(f"원본 데이터셋 크기: {original_size / (1024 * 1024):.2f} MB")
print(f"축소된 데이터셋 크기: {reduced_size / (1024 * 1024):.2f} MB")
print(f"데이터셋 축소 비율: {reduction_percentage:.2f}%")


파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000002.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000001.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000000.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000003.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000005.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000004.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000006.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000008.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000007.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000009.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000010.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000011.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000012.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000013.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_025/label/00000014.json
파일 생성됨: /home/elicer/jm/seg/대형주차장/대형주차장_

4.축소된 라벨데이터로 이미지데이터 축소

In [6]:
import os
import shutil
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jm/seg/노상'
image_source_folder = "D:/jinmin/실외/노상"
image_destination_folder = "D:/jinmin/실외/노상_seg"

# JSON 파일과 매칭되는 이미지 파일을 로컬에 복사하는 함수
def copy_matching_images(json_folder_path, image_source_folder, image_destination_folder):
    # 출력 폴더가 없으면 생성
    os.makedirs(image_destination_folder, exist_ok=True)

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)
                print(f'Processing JSON file: {json_file_path}')  # 디버깅 출력을 추가
                with open(json_file_path, 'r') as f:
                    data = json.load(f)

                    # 이미지 파일명을 JSON에서 추출하면서 모든 공백 제거
                    image_file_name = data.get("image", "").strip().replace(" ", "")
                    print(f'Extracted image file name: {image_file_name}')  # 디버깅 출력을 추가
                    
                    # 경로에서 추가적으로 필요한 정보를 붙임
                    if image_file_name:
                        # JSON 파일의 부모 디렉토리 이름 가져오기
                        json_parent_dir = os.path.basename(os.path.dirname(root))

                        # 이미지의 실제 경로 구성
                        image_source_path = os.path.join(image_source_folder, json_parent_dir, image_file_name)
                        image_destination_path = os.path.join(image_destination_folder, json_parent_dir, image_file_name)

                        print(f'Source image path: {image_source_path}')  # 디버깅 출력을 추가
                        print(f'Destination image path: {image_destination_path}')  # 디버깅 출력을 추가

                        # 이미지 파일이 존재하면 복사
                        if os.path.exists(image_source_path):
                            os.makedirs(os.path.dirname(image_destination_path), exist_ok=True)
                            shutil.copy2(image_source_path, image_destination_path)
                            print(f'이미지 파일 복사됨: {image_source_path} -> {image_destination_path}')
                        else:
                            print(f'이미지 파일이 존재하지 않음: {image_source_path}')
                    else:
                        print('Image file name is empty or not found in JSON')

# 함수 호출
copy_matching_images(json_folder_path, image_source_folder, image_destination_folder)

print("모든 이미지 파일이 성공적으로 복사되었습니다.")



Processing JSON file: /home/elicer/jm/seg/노상/노상_034/label/00000000.json
Extracted image file name: Camera/00000000.jpg
Source image path: D:/jinmin/실외/노상/노상_034/Camera/00000000.jpg
Destination image path: D:/jinmin/실외/노상_seg/노상_034/Camera/00000000.jpg
이미지 파일이 존재하지 않음: D:/jinmin/실외/노상/노상_034/Camera/00000000.jpg
Processing JSON file: /home/elicer/jm/seg/노상/노상_034/label/00000001.json
Extracted image file name: Camera/00000001.jpg
Source image path: D:/jinmin/실외/노상/노상_034/Camera/00000001.jpg
Destination image path: D:/jinmin/실외/노상_seg/노상_034/Camera/00000001.jpg
이미지 파일이 존재하지 않음: D:/jinmin/실외/노상/노상_034/Camera/00000001.jpg
Processing JSON file: /home/elicer/jm/seg/노상/노상_034/label/00000002.json
Extracted image file name: Camera/00000002.jpg
Source image path: D:/jinmin/실외/노상/노상_034/Camera/00000002.jpg
Destination image path: D:/jinmin/실외/노상_seg/노상_034/Camera/00000002.jpg
이미지 파일이 존재하지 않음: D:/jinmin/실외/노상/노상_034/Camera/00000002.jpg
Processing JSON file: /home/elicer/jm/seg/노상/노상_034/label/000000

In [9]:
import os
import shutil
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/home/elicer/jyj/실내/중소형주차장_segmentation_only'
image_source_folder = r"C:\elicetrain\088.주차공간탐색을위한차량관점복합데이터\01.데이터\1.Training\원천데이터\실내\중소형주차장"
image_destination_folder = r"C:\elicetrain\088.주차공간탐색을위한차량관점복합데이터\01.데이터\1.Training\원천데이터\실내\중소형주차장_segmentation"

# JSON 파일과 매칭되는 이미지 파일을 로컬에 복사하는 함수
def copy_matching_images(json_folder_path, image_source_folder, image_destination_folder):
    try:
        # 출력 폴더가 없으면 생성
        os.makedirs(image_destination_folder, exist_ok=True)

        for root, dirs, files in os.walk(json_folder_path):
            for file in files:
                if file.endswith('.json'):
                    json_file_path = os.path.join(root, file)
                    print(f'Processing JSON file: {json_file_path}')
                    try:
                        with open(json_file_path, 'r', encoding='utf-8') as f:  # UTF-8 인코딩으로 파일 열기
                            data = json.load(f)

                        # 이미지 파일명을 JSON에서 추출하면서 모든 공백 제거
                        image_file_name = data.get("image", "").strip().replace(" ", "")
                        print(f'Extracted image file name: {image_file_name}')
                        
                        if image_file_name:
                            # JSON 파일명에 따라 경로 결정
                            json_base_name = os.path.basename(root)
                            image_source_path = os.path.join(image_source_folder, json_base_name, "Camera", image_file_name)
                            image_destination_path = os.path.join(image_destination_folder, json_base_name, "Camera", image_file_name)

                            print(f'Source image path: {image_source_path}')
                            print(f'Destination image path: {image_destination_path}')

                            # 이미지 파일이 존재하고, 접근 가능하며, 크기 제한에 문제가 없는 경우 복사
                            if os.path.exists(image_source_path):
                                try:
                                    # 파일 권한 확인
                                    if not os.access(image_source_path, os.R_OK):
                                        raise PermissionError(f'파일 권한이 부족하여 읽을 수 없습니다: {image_source_path}')

                                    # 파일 크기 확인 (예: 1GB 제한)
                                    file_size = os.path.getsize(image_source_path)
                                    if file_size > 1 * 1024 * 1024 * 1024:
                                        raise ValueError(f'파일 크기가 너무 큽니다: {file_size / (1024 * 1024)} MB')

                                    # 파일 복사
                                    os.makedirs(os.path.dirname(image_destination_path), exist_ok=True)
                                    shutil.copy2(image_source_path, image_destination_path)
                                    print(f'이미지 파일 복사됨: {image_source_path} -> {image_destination_path}')
                                except (PermissionError, ValueError, shutil.Error) as e:
                                    print(f'파일을 복사할 수 없습니다: {e}')
                            else:
                                print(f'이미지 파일이 존재하지 않음: {image_source_path}')
                        else:
                            print('Image file name is empty or not found in JSON')
                    except (json.JSONDecodeError, UnicodeDecodeError) as e:
                        print(f'JSON 파일을 처리하는 중 오류 발생: {e}')
    except OSError as e:
        print(f'폴더를 생성하거나 접근하는 중 오류 발생: {e}')
    except Exception as e:
        print(f'예기치 못한 오류가 발생했습니다: {e}')

# 함수 호출
copy_matching_images(json_folder_path, image_source_folder, image_destination_folder)

print("모든 이미지 파일이 성공적으로 복사되었습니다.")


모든 이미지 파일이 성공적으로 복사되었습니다.
